In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import pickle
import numpy
import argparse
import os
import platform
import shutil
from tqdm import tqdm

from molzip.utils import *
from molzip.autoencoder import *

# import mdshare

In [2]:
pdb = 'pentapeptide-impl-solv.pdb'
traj = 'pentapeptide-00-500ns-impl-solv.xtc'
traj = md.load_xtc(traj, top=pdb)
traj = traj.superpose(reference=traj[0]).xyz

In [15]:
clusts = cluster_trajectory(traj, n_components=2, n_clusters=3)